In [77]:
import numpy as np

data = open("medium.in", "r").read()

data = data.split()
R = int(data[0])
C = int(data[1])
L = int(data[2])  # at least L cells of each ingredient
H = int(data[3])  # at most H cells in total
pizza = np.array([[0 if ingredient == "T" else 1 for ingredient in row] for row in data[4:]], dtype=np.int8)

In [84]:
from pulp import LpProblem, LpVariable, LpMaximize, LpAffineExpression
from sympy import divisors

def solve(pizza, L, H):
    # enumerate all possible sizes
    # min == 2*L  max == H
    sizes = []

    for n_cells in range(2 * L, H + 1):
        for div in divisors(n_cells):
            sizes.append((div, n_cells // div))
            
    # enumerate all subsets
    subsets = []
    for rows, cols in sizes:
        for i in range(pizza.shape[0] - rows + 1):
            for j in range(pizza.shape[1] - cols + 1):
                toppings = pizza[i:i+rows, j:j+cols].sum()  
                if toppings >= L and rows*cols - toppings >= L:
                    subsets.append((i, j, i+rows, j+cols))

    subsets_idx = ["%d_%d_%d_%d" % (a, b, c, d) for a, b, c, d in subsets]
    
    # constraints
    constraints = {}

    for i, s in enumerate(subsets):
        for c_i in range(s[0], s[2]):
            for c_j in range(s[1], s[3]):
                if (c_i, c_j) not in constraints:
                    constraints[(c_i, c_j)] = []
                constraints[(c_i, c_j)].append(i)    

    # chop off singletons
    constraints = {k:v for k, v in constraints.items() if len(v) > 1}
    
    # linear program
    problem = LpProblem(name="pizza", sense=LpMaximize)
    
    # variables
    slices = LpVariable.dicts("slices", subsets_idx, lowBound=0, upBound=1, cat="Binary")

    # objective
    exp = LpAffineExpression()
    for i, s in enumerate(subsets):
        exp += (s[2]-s[0])*(s[3]-s[1]) * slices[subsets_idx[i]]
    problem += exp

    # constraints
    for pos, ids in constraints.items():
        exp = LpAffineExpression()
        for i in ids:
            exp += slices[subsets_idx[i]]
        problem += exp <= 1

    # solve
    problem.solve(pulp.solvers.GLPK())
    
    # extract solution
    solution = []

    for i, s in enumerate(subsets):
        if slices[subsets_idx[i]].value() > 0.0:
            solution.append(s)
            
    return solution

def score(solution):
    return sum([(s[2]-s[0])*(s[3]-s[1]) for s in solution])

In [81]:
# Greedy approximation by solving on 20x20 sub-pizzas and by recombining solutions
solution = []
step = 20

for i in range(0, pizza.shape[0], step):
    for j in range(0, pizza.shape[1], step):
        s = solve(pizza[i:i+step, j:j+step], L, H)
        print((i, j,), cost(s))
        for piece in s:
            solution.append((piece[0]+i, piece[1]+j, piece[2]+i, piece[3]+j))

(0, 0) 400
(0, 20) 400
(0, 40) 400
(0, 60) 400
(0, 80) 400
(0, 100) 400
(0, 120) 400
(0, 140) 400
(0, 160) 400
(0, 180) 400
(0, 200) 400
(0, 220) 400
(0, 240) 200
(20, 0) 400
(20, 20) 400
(20, 40) 400
(20, 60) 400
(20, 80) 400
(20, 100) 400
(20, 120) 400
(20, 140) 400
(20, 160) 400
(20, 180) 400
(20, 200) 400
(20, 220) 400
(20, 240) 200
(40, 0) 398
(40, 20) 400
(40, 40) 400
(40, 60) 400
(40, 80) 400
(40, 100) 400
(40, 120) 400
(40, 140) 400
(40, 160) 400
(40, 180) 400
(40, 200) 400
(40, 220) 400
(40, 240) 200
(60, 0) 400
(60, 20) 400
(60, 40) 400
(60, 60) 400
(60, 80) 400
(60, 100) 400
(60, 120) 400
(60, 140) 400
(60, 160) 400
(60, 180) 400
(60, 200) 400
(60, 220) 400
(60, 240) 200
(80, 0) 400
(80, 20) 400
(80, 40) 400
(80, 60) 400
(80, 80) 400
(80, 100) 400
(80, 120) 400
(80, 140) 400
(80, 160) 400
(80, 180) 400
(80, 200) 400
(80, 220) 400
(80, 240) 200
(100, 0) 400
(100, 20) 400
(100, 40) 400
(100, 60) 400
(100, 80) 400
(100, 100) 400
(100, 120) 400
(100, 140) 400
(100, 160) 400
(100

In [85]:
print(score(solution))

49998


In [ ]:
# XXX: mark all cells from above as taken, solve for the holes 